# Generador de trayectorias (Redes aleatorias)
En esta notebook planteo un sistema de N ecuaciones diferenciales de primer orden acopladas mediante el modelo de osciladores Kuramoto que conforman una red aleatoria: 

$$ \dot{\theta_i}=\omega_i +\frac{\lambda}{\langle k\rangle}\sum_{j=1}^{N}A_{ij}\sin{(\theta_j -\theta_i)} $$

A su vez, elijo un porcentaje del total de osciladores y los perturbo con una fuerza periodica:

$$ \dot{\theta_i}=\omega_i +\frac{\lambda}{\langle k\rangle}\sum_{j=1}^{N}A_{ij}\sin{(\theta_j -\theta_i)} + b \sin{(\omega_f t - \theta_i)} $$

Donde, para redes aleatorias:

$$\langle k\rangle = p(N-1)$$

La funcion "simulator" recibe como argumentos la cantidad de osciladores N, un array con distintas fuerzas de acoplamiento $\lambda$, el metodo de integracion numerica elegido (En este caso usamos RK4), el tiempo que voy a dejar evolucionando el sistema, la intensidad de la perturbacion externa "b", la frecuencia de la perturbacion periodica externa $\omega_f$, el porcentaje de osciladores que seran perturbados p_per, la lista de vecinos de los osciladores de la red y la probabilidad de conexion $p$. El codigo simplemente devuelve la resolucion numerica del sistema de N ecuaciones diferenciales acopladas en el tiempo final $t_f$ (Se puede modificar para que devuelva la evolucion temporal completa o en el tiempo que se desee).
Dentro de la funcion simulador, se adapta la distribucion de frecuencias a datos experimentales del NSQ en donde se supuso que las celulas tiene un periodo de $23\mathrm{h}$.

In [1]:
using Plots
using LaTeXStrings
using Distributions
using JLD2
using BenchmarkTools
using Random
using Dates

## Definicion de funciones para el integrador

In [2]:
function gen_paso_rk4(N)
    k1=zeros(N)
    k2=zeros(N)
    k3=zeros(N)
    k4=zeros(N)    
    function paso_rk4!(f!,dx,x,t,h)
        # calculamos k1
        f!(k1,x,t)
        k1 .*= h
        # calculamos k2
        dx .= x .+ 0.5 .* k1 # Aca usamos dx como variable temporal
        f!(k2,dx,t+0.5*h)
        k2 .*= h
        # calculamos k3
        dx .= x .+ 0.5 .* k2
        f!(k3,dx,t+0.5*h)
        k3 .*= h
        # calculamos k4
        dx .= x .+ k3
        f!(k4,dx,t+h)
        k4 .*= h
        # calculamos dx
        dx .= x .+ (k1 .+ 2 .* k2 .+ 2 .* k3 .+ k4) ./ 6
    end
    return paso_rk4!
end

gen_paso_rk4 (generic function with 1 method)

In [3]:
# USAR ESTE INTEGRADOR
function integrador_EDO!(method!,f!,vt,vx)
    @assert size(vx)[1]>1
    for s in 2:length(vt)
        h = vt[s]-vt[s-1]
        x = view(vx,:,s-1)
        dx = view(vx,:,s)
        method!(f!,dx,x,vt[s],h)
    end
    return vx
end

integrador_EDO! (generic function with 1 method)

In [4]:
# Este integrador no devuelve la trayectoria completa, solo devuelve los puntos finales. Esto deberia agilizar el
# analisis de datos posterior. NO GARANTIZO EL FUNCIONAMIENTO CORRECTO

function integrador_EDO_rapido!(method!,f!,vt,vx)
    @assert size(vx)[1]>1
    i1,i2=1,2
    for s in 2:length(vt)
        h = vt[s]-vt[s-1]
        x = view(vx,:,i1)
        dx = view(vx,:,i2)
        method!(f!,dx,x,vt[s],h)
        i1,i2=i2,i1         
    end
    return view(vx,:,i1)
end

integrador_EDO_rapido! (generic function with 1 method)

## Generador de la red aleatoria

In [5]:
# Generador de random graph
# La funcion devuelve una lista con todas las conexiones de la red
function gen_edgelist_random(N, probabilidad_conexion)
    edgelist = []
    for i in 1:N
        for j in (i+1):N
            if rand() < probabilidad_conexion
                push!(edgelist, (i, j))
                push!(edgelist, (j, i))
            end
        end
    end
    # Ordenar la lista
    sort!(edgelist)
    edgelist_2 = zeros(Int,length(edgelist),2)
    for i in 1:length(edgelist)
        edgelist_2[i,1]=edgelist[i][1]
        edgelist_2[i,2]=edgelist[i][2]
    end
    
    return edgelist_2
end

gen_edgelist_random (generic function with 1 method)

## Implementacion del simulador

In [6]:
# Para simular genero una funcion que lo haga todo. Le paso el integrador que voy a usar, la cantidad de osciladores "N", una lista con valores de λ y vector "vt" que tiene
# el rango de tiempos en el cual voy a trabajar.
#
# Inputs:
#       integrador!: El integrador que se usara para el sistema de ecuaciones
#       N: Cantidad de osciladores
#       array_λ: Valores de la fuerza de acoplamiento a utilizar
#       vt: El tiempo que dejo evolucionar al sistema
#       p: Probabilidad de conexion de la red
#       edgelist: Lista de vecinos de la red 
#       b: Intensidad de la fuerza periodica pertubativa
#       ω_f: Frecuencia de la fuerza periodica perturbativa
#       p_pert: Porcentaje del total de osciladores a ser perturbados
 
# Return: 
#         Esta celda devuelve un array 1x7: 
#               [b: Intensidad de la perturbacion,
#                N: Cantidad de osciladores, 
#                λ: Intensidad de acoplamiento,
#                vt: Vector tiempo,
#                vθ: Trayectorias(N x len(vt)),
#                indices_perturbar: array con los osciladores que fueron perturbados,
#                ω: Frecuencias intrinsecas de los osciladores]

function simulation(integrador!, N, array_λ, vt, p, edgelist, b, ω_f, p_pert)
    paso_rk4! = gen_paso_rk4(N)
    vθ = zeros(N,length(vt))
    ω = zeros(N)
    
    # Defino los osciladores que voy a perturbar
    N_per = Int(round(N*p_pert)) # Cantidad de osciladores perturbados

    muestras = []
    for λ in array_λ
        
        # Sorteo los indices de los osciladores que voy a perturbar
        indices_perturbar = sample(1:1:N, N_per, replace=false) 

        function f_kuramoto!(dθ,θ,t) # Defino los osciladores
            i_anterior = 1
            suma = 0
            for e in 1:size(edgelist)[1]
                i=edgelist[e,1]
                j=edgelist[e,2]
                if i-i_anterior > 1 # Chequear si salteamos algun indice (Indicador de oscilador/es desacoplado/s)
                    indice = i-1 # Agrego los osciladores desacoplados a la lista
                    while indice > i_anterior
                        dθ[indice] = ω[indice]
                        indice = indice -1
                    end
                end
                if i_anterior != i
                    dθ[i_anterior] = ω[i_anterior]+(λ/(p*(N-1)))*suma
                    i_anterior = i
                    suma = 0
                end
                suma += sin(θ[j]-θ[i])
                if e == size(edgelist)[1]
                    dθ[i_anterior]= ω[i_anterior] + (λ/(p*(N-1)))*suma
                    index = i
                    while index < N
                        dθ[index+1] = ω[index+1]
                        index = index + 1
                    end
                end
            end
            if size(edgelist)[1] == 0 # Contingencia para el caso en que no haya NINGUN nodo conectado
                for j in 1:N
                    dθ[j] = ω[j]
                end
            end
            
            # Ahora perturbo con la fuerza periodica externa
            for k in 1:length(indices_perturbar)
                index = indices_perturbar[k]
                dθ[index] += b*sin(ω_f*t - θ[index])
            end
        end
        # Condiciones iniciales
        vθ[:,1] .= rand(Truncated(Normal(π,2), 0, 2π), N)
        # Frecuencias adaptadas a datos experimentales del NSQ
        μ = 0.2739
        σ = 0.0122
        ω = rand(Normal(μ,σ), N)
        # Integrador
        muestra = integrador!(paso_rk4!,f_kuramoto!,vt,vθ)
        # Almaceno los datos de las simulaciones correspondientes a t=t_f
        push!(muestras,[b,N,λ,vt,copy(muestra[:,end]),indices_perturbar,ω])
    end
    return muestras
end

simulation (generic function with 1 method)

In [8]:
n_muestras = 10 # Cantidad de muestras por red
n_redes = 1 # Cantidad de redes
trayectorias = Array{Any}(undef, n_muestras, n_redes)

N = 1024 # Tamaño
p = 0.0059 # Probabilidad de conexion

b = 0.1 # Intensidad de la perturbacion
ω_z = 0.262 # Frecuencia del zeitgeber
p_per = 0.1 # Porcentaje de osciladores perturbados

λ = 0:0.01:0.4 # λs recorridos
t = 0.0:2:2000 # Tiempo de integracion

@time for m in 1:n_redes
    edgelist = gen_edgelist_random(N,p)
    task = Threads.@spawn begin
        Threads.@threads for n in 1:n_muestras
            trayectorias[n,m] = simulation(integrador_EDO!, N, λ, t, p, edgelist, b, ω_z, p_per)
        end
    end
    wait(task)
end

  3.650727 seconds (8.55 M allocations: 451.816 MiB, 4.85% gc time, 99.00% compilation time)


In [9]:
# Aca cargo o genero un diccionario sobre el cual almacenar los datos
data = Dict()
try
    # We load the data dictionary if it exists
    global data = load("RN Perturbado "*string(p_per*100)*"% b="*string(b)*" N="*string(N)*".jld2")
catch
    # Otherwise we create the dictionary
    println("Estas usando un diccionario vacio")
end

Estas usando un diccionario vacio


In [10]:
# Almaceno los resultados en el diccionario usando la fecha actual como llave del mismo
data["Red ER p="*string(p)*"b="*string(b)*" N="*string(N)*" "*string(p_per*100)*"% perturbado - "*Dates.format(Dates.now(),"yyyy-mm-dd-HH-MM-SS")]=trayectorias;

In [22]:
# Guardo un archivo con los datos para su posterior analisis.
save("RN Perturbado "*string(p_per*100)*"% b="*string(b)*" N="*string(N)*".jld2",data)